<a href="https://colab.research.google.com/github/grace3999/PPs/blob/master/4_FIFI_image_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#mount google drive containings required files: 1) csv of annotation features, 2) netcdf files
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import numpy as np
import pandas as pd
import os
import requests 
from multiprocessing.pool import ThreadPool

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)

import warnings; warnings.simplefilter('ignore')
np.set_printoptions(suppress=True)

In [0]:
#path for FIFI cleaned and preprocessed data frame
path_FIFI = '/content/gdrive/My Drive/WIDS_FIFI_groupproject/generated_datasets/data_final.pkl'

In [4]:
#read in pkl file containing 
data = pd.read_pickle(path_FIFI)
data = pd.DataFrame(data = data)
data.reset_index(inplace=True, drop=True)

print('Original data shape:\n', data.shape, '\n')
print('Original data columns:\n', data.columns.values, '\n')

data = data[data['FIFI_category'] != 'Community Walk']

data['FIFI_category'] = data['FIFI_category'].replace({'Needles/Dumping': 'Needles_Dumping', 'Sign/Signal': 'Sign_Signal'})

data.head(1)

Original data shape:
 (248815, 67) 

Original data columns:
 ['Service_Request_Number' 'Created_Date' 'Location' 'Location_Details'
 'Description' 'License_Number' 'State' 'Vehicle_Make' 'Vehicle_Color'
 'Time_parked' 'Photo' 'FIFI_category' 'Location_Details.1'
 'Property_damage?' 'Current_Issue' 'Type_of_animal'
 'On_private_property?' 'Graffiti_location' 'Location_and_Description'
 'Obscene/Racial/Hateful?' 'Street/Intersection' 'Where?' 'Type_of_item?'
 'Veg_location' 'Vegetation_Safety_Issue' 'Parking_Violation_Concerning'
 'License_Plate_Number' 'Location_details' '7_digit_number_on_pole'
 'Issue' 'date' 'year' 'month' 'day' 'day_of_week' 'hour' 'zip'
 'neighborhood' 'SittingInformal' 'Lying' 'DisruptiveActivity_Aggressive'
 'DisruptiveActivity_Intoxicated' 'LivingActivities' 'Soliciting'
 'overdose' 'pop' 'pop_dens' 'housing_units' 'med_home' 'land_area'
 'occupied_units' 'occupied_ratio' 'med_income' 'med_age' 'male_%'
 'white_%' 'fam_%' 'nokid_%' 'mortgage_%' 'fulltime_%' 'une

,Service_Request_Number,Created_Date,Location,Location_Details,Description,License_Number,State,Vehicle_Make,Vehicle_Color,Time_parked,Photo,FIFI_category,Location_Details.1,Property_damage?,Current_Issue,Type_of_animal,On_private_property?,Graffiti_location,Location_and_Description,Obscene/Racial/Hateful?,Street/Intersection,Where?,Type_of_item?,Veg_location,Vegetation_Safety_Issue,Parking_Violation_Concerning,License_Plate_Number,Location_details,7_digit_number_on_pole,Issue,date,year,month,day,day_of_week,hour,zip,neighborhood,SittingInformal,Lying,DisruptiveActivity_Aggressive,DisruptiveActivity_Intoxicated,LivingActivities,Soliciting,overdose,pop,pop_dens,housing_units,med_home,land_area,occupied_units,occupied_ratio,med_income,med_age,male_%,white_%,fam_%,nokid_%,mortgage_%,fulltime_%,unemploy,drive_%,publictrans_%,other_%,wrk_travel,city council,county council
0,13-00036779,2013-05-09 20:32:33,"5208 BEACON AVE S, SEATTLE, WA","5200 Beacon Ave S Seattle, WA 98108; XY: 1277537.1732614825, 206078.498081425; LatLng: 47.555513, -122.302999",Ford light blue,AHZ9829,NaN,Ford,Blue,Over 3 Days,NaN,Abnd_Vehicle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-05-09,2013,5,9,Thursday,20,WA,WA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#we are only interested in FIFI requests that have a photo included with the request
print(data.shape)
data_photo = data[~data['Photo'].isna()]
print(data_photo.shape)
data_photo.head(1)

(248661, 67)
(183278, 67)


,Service_Request_Number,Created_Date,Location,Location_Details,Description,License_Number,State,Vehicle_Make,Vehicle_Color,Time_parked,Photo,FIFI_category,Location_Details.1,Property_damage?,Current_Issue,Type_of_animal,On_private_property?,Graffiti_location,Location_and_Description,Obscene/Racial/Hateful?,Street/Intersection,Where?,Type_of_item?,Veg_location,Vegetation_Safety_Issue,Parking_Violation_Concerning,License_Plate_Number,Location_details,7_digit_number_on_pole,Issue,date,year,month,day,day_of_week,hour,zip,neighborhood,SittingInformal,Lying,DisruptiveActivity_Aggressive,DisruptiveActivity_Intoxicated,LivingActivities,Soliciting,overdose,pop,pop_dens,housing_units,med_home,land_area,occupied_units,occupied_ratio,med_income,med_age,male_%,white_%,fam_%,nokid_%,mortgage_%,fulltime_%,unemploy,drive_%,publictrans_%,other_%,wrk_travel,city council,county council
1,13-00064098,2013-08-13 18:05:17,"2629 SW ANDOVER ST, SEATTLE, WA","26012799 SW Andover St, Seattle; XY: 1261975.9119158287, 211063.7734459653; LatLng: 47.56828403207891, -122.3664187277485",NaN,Acxo217,NaN,Suzuki,Other,Over 1 Week,http://servicerequest.seattle.gov/media/seattle/report/photos/520ad788016358ed6cc31006/report.jpg,Abnd_Vehicle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-08-13,2013,8,13,Tuesday,18,WA,WA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
data_photo['FIFI_category'].value_counts()

Needles_Dumping    40265
Other              34428
Grafitti           31749
Abnd_Vehicle       26656
Parking            25140
Pothole            13154
Sign_Signal        8455 
Streetlight        2943 
Clogged_Drain      427  
Vegetation         44   
Dead Animal        17   
Name: FIFI_category, dtype: int64

In [7]:
#create a directory for each category
save_path = '/content/gdrive/My Drive/WIDS_FIFI_groupproject/photos_by_category/'

cats_to_dl = ['Abnd_Vehicle', 'Grafitti', 'Needles_Dumping', 'Parking',
       'Pothole', 'Streetlight', 'Other', 'Clogged_Drain', 'Dead Animal',
       'Vegetation', 'Sign_Signal']

#get and save photos according to FIFI_category
for cat in cats_to_dl:
    if os.path.exists(os.path.join(save_path, cat)) == True:
      print(cat, " already exists")
    else:
      os.mkdir(os.path.join(save_path, cat))
      print(cat, " directory created")

Abnd_Vehicle  directory created
Grafitti  directory created
Needles_Dumping  directory created
Parking  already exists
Pothole  already exists
Streetlight  already exists
Other  already exists
Clogged_Drain  already exists
Dead Animal  already exists
Vegetation  already exists
Sign_Signal  already exists


In [21]:
data_photo[data_photo['FIFI_category'] == 'Parking'].iloc[52]['Photo']

'http://servicerequest.seattle.gov/media/seattle/report/photos/52153903016358ed6cc351ce/report.jpg'

In [17]:
os.listdir(save_path_cat)[-100:]

['Streetlight_1063.jpeg',
 'Streetlight_1049.jpeg',
 'Streetlight_1011.jpeg',
 'Streetlight_1066.jpeg',
 'Streetlight_1002.jpeg',
 'Streetlight_1033.jpeg',
 'Streetlight_988.jpeg',
 'Streetlight_1008.jpeg',
 'Streetlight_1043.jpeg',
 'Streetlight_1034.jpeg',
 'Streetlight_1003.jpeg',
 'Streetlight_1050.jpeg',
 'Streetlight_991.jpeg',
 'Streetlight_989.jpeg',
 'Streetlight_1020.jpeg',
 'Streetlight_992.jpeg',
 'Streetlight_1019.jpeg',
 'Streetlight_999.jpeg',
 'Streetlight_1075.jpeg',
 'Streetlight_990.jpeg',
 'Streetlight_1026.jpeg',
 'Streetlight_1062.jpeg',
 'Streetlight_1014.jpeg',
 'Streetlight_1035.jpeg',
 'Streetlight_1044.jpeg',
 'Streetlight_998.jpeg',
 'Streetlight_1022.jpeg',
 'Streetlight_1031.jpeg',
 'Streetlight_1078.jpeg',
 'Streetlight_1051.jpeg',
 'Streetlight_1072.jpeg',
 'Streetlight_1028.jpeg',
 'Streetlight_1000.jpeg',
 'Streetlight_1057.jpeg',
 'Streetlight_1015.jpeg',
 'Streetlight_1023.jpeg',
 'Streetlight_1013.jpeg',
 'Streetlight_1073.jpeg',
 'Streetlight_997.j

In [18]:
save_path = '/content/gdrive/My Drive/WIDS_FIFI_groupproject/photos_by_category/'

cats_to_dl = ['Abnd_Vehicle', 'Grafitti', 'Needles_Dumping', 'Parking',
       'Pothole', 'Streetlight', 'Other', 'Clogged_Drain', 'Dead Animal',
       'Vegetation', 'Sign_Signal']
cat = cats_to_dl[5]
print(cat)
save_path_cat = os.path.join(save_path, cat)
len(os.listdir(save_path_cat))

Streetlight


2943

In [8]:
#create function to get photo from url and save

def fetch_and_save(image_url_tupple):
    image_name, image_url = image_url_tupple
    #create file name for photo
    image_name = os.path.join(save_path_cat, image_name)  
    #get photo from url and save
    r = requests.get(image_url, allow_redirects=True)
    open(image_name, 'wb').write(r.content)

#do each category individually and 5,000 at a time (runtime keeps dying)
save_path = '/content/gdrive/My Drive/WIDS_FIFI_groupproject/photos_by_category/'

cats_to_dl = ['Abnd_Vehicle', 'Grafitti', 'Needles_Dumping', 'Parking',
       'Pothole', 'Streetlight', 'Other', 'Clogged_Drain', 'Dead Animal',
       'Vegetation', 'Sign_Signal']

cat = cats_to_dl[5]
print(cat)
cat_data = data_photo[data_photo['FIFI_category'] == cat].reset_index() #index will get saved as part of file name for later use

save_path_cat = os.path.join(save_path, cat)

i = 0
while i < (5000):
  if i%50 == 0:
    print(i)
  urls = list(zip([cat+'_'+str(x)+'.jpeg' for x in cat_data.index.values[i:i+5]], cat_data['Photo'][i:i+5]))
  ThreadPool(5).imap_unordered(fetch_and_save, urls)
  i += 5

Streetlight
0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4700
4750
4800
4850
4900
4950
